In [22]:
import pandas as pd

In [23]:
diagnosis_df = pd.read_csv('../data/raw/trainingSet/training_SyncDiagnosis.csv')
patient_df = pd.read_csv('../data/raw/trainingSet/training_SyncPatient.csv')
transcript_df = pd.read_csv('../data/raw/trainingSet/training_SyncTranscript.csv')
conversion_df = pd.read_csv('../data/raw/trainingSet/training_SyncTranscriptDiagnosis.csv')

patient_df.head()

,PatientGuid,DMIndicator,Gender,YearOfBirth,State,PracticeGuid
0,FB6EFC3D-1A20-4497-9CBD-00027CC5D220,0,M,1929,SD,7BF4DAD8-5F67-4985-B911-20C9E89A3737
1,C6746626-6783-4650-A58F-00065649139A,0,F,1985,TX,E7101967-2FF1-4B0F-8129-B0B429D1D15C
2,E05C6E8F-779F-4594-A388-000C635AE4D3,0,F,1984,NJ,FC01A799-1CAF-464F-A86F-8A666AB86F32
3,EAEBD216-F847-4355-87B2-000D942E08F0,0,M,1959,OH,EEBC95EF-79BE-4542-892E-98D3166BAB20
4,C7F10A80-4934-42D2-8540-000FBEBA75C8,0,F,1990,FL,677BA32E-B4C4-48F2-86E4-08C42B135401


In [70]:
transcript_df.VisitYear.value_counts()

2010    48132
2011    43360
2009    19945
2012    15640
0        3954
Name: VisitYear, dtype: int64

In [25]:
def dense_dataframe(df, on):
    """dense the dataframe by combining entries of same value of a given column"""
    lst = []
    for index, value in df.iteritems(): # loop through columns in the dataframe
        # combines rows of values of each column to a list, and store the lists into a list
        lst.append(df.groupby(on)[index].apply(list)) 
        
    processed_df = pd.concat(lst, axis=1) # convert the list of lists of values to a dataframe 
    processed_df[on] = processed_df[on].apply(lambda x:x[0]) # retrieve the same value of 'on' column from the list    
    return processed_df

processed_conversion_df = dense_dataframe(conversion_df, 'TranscriptGuid')
processed_transcripts_df = dense_dataframe(transcript_df, 'PatientGuid')
# pd.merge(processed_diagnosis_df, processed_transcripts_df, 'TranscriptGuid')
# processed_transcript_df = dense_dataframe(transcript_df, 'PatientGuid')
# transcript_diagnosis_df = processed_transcript_df.merge(processed_diagnoses_df, 'TranscriptGuid')
# patient_transcript_df = patient_df.merge(processed_transcript_df, on='PatientGuid')
# patient_transcript_diagnosis_df = patient

In [30]:
conversion_dict1 = processed_conversion_df.set_index('TranscriptGuid')['DiagnosisGuid'].to_dict()
conversion_dict2 = diagnosis_df.set_index('DiagnosisGuid').to_dict()

In [72]:
def get_diagnoses(transcripts):
    diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all = [], [], [], [], [], []

    for i in transcripts:
        if i in conversion_dict:
            diag = conversion_dict[i]
            diagnoses.append(diag)

            descriptions_one, acute_one, icd9_one, start_one, stop_one = [], [], [], [], []
            for k in diag:
                descriptions_one.append(conversion_dict2['DiagnosisDescription'][k])
                acute_one.append(       conversion_dict2['Acute'][k])
                icd9_one.append(        conversion_dict2['ICD9Code'][k])
                start_one.append(       conversion_dict2['StartYear'][k])
                stop_one.append(        conversion_dict2['StopYear'][k])
            
            descriptions_all.append(descriptions_one)
            acute_all.append(acute_one)
            icd9_all.append(icd9_one)
            start_all.append(start_one)
            stop_all.append(stop_one)
            
    return diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all

def combine_data(conversion, transcripts):
    """ Matches diagnosis data with each transcript """
    
    transcripts['CombinedData'] = transcripts.TranscriptGuid.apply(get_diagnoses)
    
    transcripts['DiagnosisGuid']        = [x[0] for x in transcripts['CombinedData']]
    transcripts['DiagnosisDescription'] = [x[1] for x in transcripts['CombinedData']]
    transcripts['Acute']                = [x[2] for x in transcripts['CombinedData']]
    transcripts['ICD9Code']             = [x[3] for x in transcripts['CombinedData']]
    transcripts['StartYear']            = [x[4] for x in transcripts['CombinedData']]
    transcripts['StopYear']             = [x[5] for x in transcripts['CombinedData']]
        
    return transcripts
        
combined_data = combine_data(processed_conversion_df, processed_transcripts_df)
combined_data.head()

,TranscriptGuid,PatientGuid,VisitYear,Height,Weight,BMI,SystolicBP,DiastolicBP,RespiratoryRate,HeartRate,...,UserGuid,AllDiagnoses,Diagnosis,CombinedData,DiagnosisDescription,DiagnosisGuid,Acute,ICD9Code,StartYear,StopYear
PatientGuid,,,,,,,,,,,,,,,,,,,,,
00023761-9D8D-445B-874C-2424CC7CF620,"[C024B9A8-C201-4013-9DD4-16877BFCBDCF, A04B906...",00023761-9D8D-445B-874C-2424CC7CF620,"[2009, 2009, 2009]","[68.0, 67.0, 67.0]","[182.0, 184.0, 185.0]","[27.67, 28.815, 28.972]","[122.0, 134.0, 120.0]","[76.0, 86.0, 62.0]","[16.0, nan, nan]","[nan, nan, nan]",...,"[5A5D58CC-8874-4DCB-8E38-ED2B89FAC111, A05B47B...","[[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D0...","([[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D...","([[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D...",[[Hypertrophy (Benign) of prostate ...,"[[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D0...","[[0, 0], [0, 0, 0], [0, 0]]","[[600.0, 272.2], [600.0, 909.5, 272.2], [600.0...","[[0, 0], [0, 0, 0], [0, 0]]","[[nan, nan], [nan, nan, nan], [nan, nan]]"
0005D9BD-0247-4F02-B7EE-7C1B44825FA1,"[CC8E82FF-2588-4D24-878A-05D0A7A82C68, 0BF992A...",0005D9BD-0247-4F02-B7EE-7C1B44825FA1,"[2009, 2010, 0, 0, 0, 0, 2009, 2010, 0, 2010, ...","[nan, 65.0, nan, 65.0, 65.0, 65.0, nan, nan, 6...","[0.0, 135.0, 0.0, 137.0, 137.0, 137.0, 0.0, 0....","[0.0, 22.463, 0.0, 22.796, 22.796, 22.796, 0.0...","[135.0, 130.0, nan, 128.0, 140.0, 133.0, 125.0...","[75.0, 80.0, nan, 60.0, 59.0, 73.0, 78.0, 82.0...","[nan, nan, nan, nan, nan, nan, 15.0, nan, nan,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[901ABAB2-B8EB-4EE2-92B5-3F1A5DCB13AE, 1B9AED5...","[[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852E...","([[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852...","([[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852...","[[Mixed hyperlipidemia, Other psoriasis and si...","[[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852E...","[[0, 0, 0], [0], [0, 0], [0, 0, 0], [0, 0, 0, 0]]","[[272.2, 696.1, 789.06], [696.1], [696.1, 272....","[[2009, 0, 2009], [0], [0, 2009], [0, 0, 2009]...","[[nan, nan, nan], [nan], [nan, nan], [nan, nan..."
000B4862-7CE7-4EC5-8043-A97FCD74BD78,"[582D3C53-0B68-4B68-96FA-0D5664D32392, B0D082A...",000B4862-7CE7-4EC5-8043-A97FCD74BD78,"[2012, 2012, 2011, 2011, 2012, 2012, 2012, 201...","[65.0, nan, nan, nan, nan, nan, nan, nan, nan,...","[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[16.639, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[100.0, 80.0, 83.0, nan, nan, nan, nan, nan, 1...","[60.0, 60.0, 56.0, nan, nan, nan, nan, nan, 60...","[18.0, 14.0, 14.0, nan, nan, nan, nan, nan, 20...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[EE1410C4-FF22-4BE3-9585-45E354C90341, 263A145...","[[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8C...","([[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8...","([[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8...","[[Impacted cerumen, Urinary incontinence, unsp...","[[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8C...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0...","[[380.4, 788.30, 781.2, 458.9, 428.0, 783.0, 4...","[[0, 2012, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0],...","[[nan, nan, nan, nan, nan, nan, nan], [nan, na..."
00110ABC-DAB9-49E3-A1C8-88BBF8D58109,"[DD03810C-ADF4-4B0D-B43D-01F9FCE71F55, 0393555...",00110ABC-DAB9-49E3-A1C8-88BBF8D58109,"[2012, 2011, 2012, 2011, 2009, 2010, 2012, 201...","[nan, nan, nan, nan, 71.0, nan, nan, nan, nan,...","[0.0, 0.0, 0.0, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 29.983, 0.0, 0.0, 0.0, 0....","[nan, nan, nan, nan, 130.0, nan, nan, nan, nan...","[nan, nan, nan, nan, 80.0, nan, nan, nan, nan,...","[nan, nan, nan, nan, 12.0, nan, nan, nan, nan,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[62C7E69B-EB1E-4711-A6D7-7AB644BEB1F9, 62C7E69...","[[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9BE...","([[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9B...","([[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9B...","[[Sebaceous cyst], [Other enthesopathy of ankl...","[[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9BE